# Statistical Language Modeling

- Language Understanding Systems
- Evgeny A. Stepanov
- stepanov.evgeny.a@gmail.com

## Introduction

This notebook is part of the Laboratory Work for [Language Understanding Systems class](http://disi.unitn.it/~riccardi/page7/page13/page13.html) of [University of Trento](https://www.unitn.it/en).
Laboratory has been ported to jupyer notebook format for remote teaching during [COVID-19 pandemic](https://en.wikipedia.org/wiki/2019%E2%80%9320_coronavirus_pandemic).

This notebook covers Lecture on __Sequence and Language Modeling__.

Dan Jurafsky and James H. Martin's __Speech and Language Processing__ ([3rd ed. draft](https://web.stanford.edu/~jurafsky/slp3/)) is adviced for reading. 

- Section *Corpora and Counting* covers some conceptes of *Chapter 2: "Regular Expressions, Text Normalization, Edit Distance"*.
- Sections *Ngrams and Ngram Probabilites* and *Language Models* roughly cover *Chapter 3: "N-gram Language Models"*. 

<span style="color: blue;">__NOTE__</span>: 

Due to popularity of Python as a programming language for Natural Langauge Processing and Machine Learning, the exercises use Python (occassionally Unix Shell).
Prefer to avoid using libraries, unless exlicitly stated (e.g. don't use [`Counter`](https://docs.python.org/3/library/collections.html#collections.Counter)). 


## Requirements

- [NL2SparQL4NLU](https://github.com/esrel/NL2SparQL4NLU) dataset

    - run `git clone https://github.com/esrel/NL2SparQL4NLU.git`
    

## 1. Corpora and Counting

### 1.1. Corpus

[Corpus](https://en.wikipedia.org/wiki/Text_corpus) is a collection of written or spoken texts that is used for language research. Before doing anything with a corpus we need to know its properties:

__Corpus Properties__:
- *Format* -- how to read/load it?
- *Language* -- which tools/models can I use?
- *Annotation* -- what it is intended for?
- *Split* for __Evaluation__: (terminology varies from source to source)

| Set         | Purpose                                       |
|:------------|:----------------------------------------------|
| Training    | training model, extracting rules, etc.        |
| Development | tuning, optimization, intermediate evaluation |
| Test        | final evaluation (remains unseen)             |


#### NL2SparQL4NLU

- __Format__:

    - Utterance (sentence) per line
    - Tokenized
    - Lowercased

- __Language__: English monolingual

- __Annotation__: None (for now)

- __Split__: training & test sets

#### Exercise

- define a function to load a corpus into a list-of-lists

- load `NL2SparQL4NLU/dataset/NL2SparQL4NLU.train.utterances.txt`
- print first `2` words (tokens) of the first `10` sentences


In [1]:
trn='NL2SparQL4NLU/dataset/NL2SparQL4NLU.train.utterances.txt'
tst='NL2SparQL4NLU/dataset/NL2SparQL4NLU.test.utterances.txt'

### 1.2. Corpus Descriptive Statistics (Counting)

*Corpus* description in terms of:

- total number of words
- total number of utterances


#### Exercise

- define a function to compute corpus descriptive statistics -- total utterance and word counts.
- compute the statistics for the __training__ and __test__ sets of NL2SparQL4NLU dataset. 
- compare the computed statistics with the reference values below.


| Metric           | Train  | Test   |
|------------------|-------:|-------:|
| Total Words      | 21,453 |  7,117 |
| Total Utterances |  3,338 |  1,084 |


### 1.3. Lexicon

[Lexicon](https://en.wikipedia.org/wiki/Lexicon) is the *vocabulary* of a language. In linguistics, a lexicon is a language's inventory of lexemes.

Linguistic theories generally regard human languages as consisting of two parts: a lexicon, essentially a catalogue of a language's words; and a grammar, a system of rules which allow for the combination of those words into meaningful sentences. 

*Lexion (or Vocabulary) Size* is one of the statistics reported for corpora. While *Word Count* is the number of __tokens__, *Lexicon Size* is the number of __types__ (unique words).


#### Exercise

- define a function to compute a lexicon from corpus in a list-of-lists format
    - sort the list alphabetically
    
- compute the lexicon of the training set of NL2SparQL4NLU dataset
- compare the its size to the reference value below.

| Metric       | Value |
|--------------|------:|
| Lexicon Size | 1,729 |


### 1.4. Frequency List

In Natural Language Processing (NLP), [a frequency list](https://en.wikipedia.org/wiki/Word_lists_by_frequency) is a sorted list of words (word types) together with their frequency, where frequency here usually means the number of occurrences in a given corpus, from which the rank can be derived as the position in the list.

What is a "word"?

- case sensitive counts
- case insensitive counts (our corpus is lowercased)

#### Exercise

- define a function to compute a frequency list for a corpus
- compute frequency list for the training set of NL2SparQL4NLU dataset
- report `5` most frequent words (use can use provided `nbest` function to get a dict of top N items)
- compate the frequencies to the reference values below

| Word   | Frequency |
|--------|----------:|
| the    |     1,337 |
| movies |     1,126 |
| of     |       607 |
| in     |       582 |
| movie  |       564 |


In [2]:
def nbest(d, n=1):
    """
    get n max values from a dict
    :param d: input dict (values are numbers)
    :param n: number of values to get (int)
    :return: dict of top n key-values
    """
    return dict(sorted(d.items(), key=lambda item: item[1], reverse=True)[:n])

### 1.5. Lexicon Operations

It is common to process the lexicon according to the task at hand (not every transformation makes sense for all tasks). The common operations are removing words by frequency (minimum or maximum, i.e. *Frequency Cut-Off*) and removing words for a specific lists (i.e. *Stop Word Removal*).

In computing, [stop words](https://en.wikipedia.org/wiki/Stop_words) are words which are filtered out before or after processing of natural language data (text). Though "stop words" usually refers to the most common words in a language, there is no single universal list of stop words used by all natural language processing tools, and indeed not all tools even use such a list. Some tools specifically avoid removing these stop words to support phrase search.

Any group of words can be chosen as the stop words for a given purpose.

#### Exercises

##### Frequency Cut-Off

- define a function to compute a lexicon from a frequency list applying minimum and maximum frequency cut-offs

    - use default values for min and max
    
- using frequency list for the training set of NL2SparQL4NLU dataset
    
    - compute lexicon applying:
    
        - minimum cut-off 2 (remove words that appear less than 2 times, i.e. remove [hapax legomena](https://en.wikipedia.org/wiki/Hapax_legomenon))
        - maximum cut-off 100 (remove words that appear more that 100 times)
        - both minimum and maximum thresholds together
        
    - report size for each comparing to the reference values in the table

| Operation  | Min | Max | Size |
|------------|----:|----:|-----:|
| original   | N/A | N/A | 1729 |
| cut-off    |   2 | N/A |  950 |
| cut-off    | N/A | 100 | 1694 |
| cut-off    |   2 | 100 |  915 |


##### Stop Word Removal

- define a function to read/load a list of words in token-per-line format (i.e. lexicon)
- load stop word list from `NL2SparQL4NLU/extras/english.stop.txt`
- using Python's built it `set` [methods](https://docs.python.org/2/library/stdtypes.html#set):
    
    - define a function to compute overlap of two lexicons
    - define a function to apply a stopword list to a lexicon

- compare the 100 most frequent words in frequency list of the training set to the list of stopwords (report count)
- apply stopword list to the lexicon of the training set
- report size of the resulting lexicon comparing to the reference values.

| Operation       | Size |
|-----------------|-----:|
| original        | 1729 |
| no stop words   | 1529 |
| top 100 overlap |   50 |

In [3]:
swl='NL2SparQL4NLU/extras/english.stop.txt'

### <span style="color: blue;">Solutions</span>

#### Function Definitions

In [4]:
def read_corpus(corpus_file):
    """
    read corpus into a list-of-lists, splitting sentences into tokens by space (' ')
    :param corpus_file: corpus file in sentence-per-line format (tokenized)
    :return: corpus as list of lists
    """
    return [line.strip().split() for line in open(corpus_file, 'r')]

def read_lexicon(lexicon_file):
    """
    read lexicon into a list
    :param lexicon_file: lexicon file in token-per-line format
    :return: lexicon as a list
    """
    return [line.strip() for line in open(lexicon_file, 'r')]

def compute_lexicon(corpus):
    """
    compute lexicon of a corpus
    :param corpus: corpus as list-of-lists
    :return: sorted list of unique words
    """
    return sorted(list(set([word for sent in corpus for word in sent])))

def compute_frequency_list(corpus):
    """
    create frequency list for a corpus
    :param corpus: corpus in list-of-lists format
    :return: frequency list as dict of counts
    """
    frequencies = {}
    for sentence in corpus:
        for token in sentence:
            frequencies[token] = frequencies.setdefault(token, 0) + 1 
    return frequencies

def nbest(d, n=1):
    """
    get n top values from a dict
    :param d: input dict (values are numbers)
    :param n: number of values to get (int)
    :return: dict of top n key-values
    """
    return dict(sorted(d.items(), key=lambda item: item[1], reverse=True)[:n])

def cutoff(frequency_list, tf_min=1, tf_max=float('inf')):
    """
    apply min and max cutoffs to a frequency list
    :param frequency_list: frequency list of a corpus as dict
    :param tf_min: minimum token frequency for lexicon elements (below removed); default 1
    :param tf_max: maximum token frequency for lexicon elements (above removed); default infinity
    :return: lexicon as sorted list of tokens
    """
    return sorted([token for token, frequency in frequency_list.items() if tf_max >= frequency >= tf_min])

def compute_overlap(a, b):
    """
    compute overal of two lists as set intersection
    :param a: list 1
    :param b: list 2
    :return: sorted list of overlapping elements
    """
    return sorted(list(set(a) & set(b)))
    
def remove_stopwords(lexicon, stopwords):
    """
    remove stopwords from a lexicon
    :param lexicon: lexicon as a list
    :param stopwords: stopwords list
    :return: sorted difference of two lists (lexicon - stopwords)
    """
    return sorted(list(set(lexicon) - set(stopwords)))

def corpus_stats(corpus):
    """
    compute word and sentence counts of the corpus
    :param corpus: corpus as list-of-lists
    :return: sentence count, word count
    """
    return len(corpus), sum([len(sent) for sent in corpus])

#### Set paths to data set files

In [5]:
trn = 'NL2SparQL4NLU/dataset/NL2SparQL4NLU.train.utterances.txt'
tst = 'NL2SparQL4NLU/dataset/NL2SparQL4NLU.test.utterances.txt'
swl = 'NL2SparQL4NLU/extras/english.stop.txt'

#### Corpus Reading & Iterating

In [6]:
corpus = read_corpus(trn)

for sentence in corpus[:10]:
    print(sentence[:2])

['who', 'plays']
['show', 'credits']
['who', 'was']
['find', 'the']
['who', 'played']
['who', 'was']
['who', 'played']
['who', 'was']
['find', 'the']
['cast', 'and']


#### Corpus Statistics

In [7]:
print(corpus_stats(read_corpus(trn)))
print(corpus_stats(read_corpus(tst)))

(3338, 21453)
(1084, 7117)


#### Computing Lexicon & Its Size

In [8]:
lex = compute_lexicon(corpus)
print(len(lex))

1729


#### Computing Frequency List

In [9]:
freq_list = compute_frequency_list(corpus)
print(nbest(freq_list, n=5))

{'the': 1337, 'movies': 1126, 'of': 607, 'in': 582, 'movie': 564}


#### Applying Cut-Off & Reporting Lexicon Sizes

In [10]:
print(len(cutoff(freq_list)))
print(len(cutoff(freq_list, tf_min=2)))
print(len(cutoff(freq_list, tf_max=100)))
print(len(cutoff(freq_list, tf_min=2, tf_max=100)))

1729
950
1694
915


#### Computing Lexicon Overlap (Set Intersection)

In [11]:
stopwords = read_lexicon(swl)
top_words = list(nbest(freq_list, n=100).keys())
print(len(compute_overlap(top_words, stopwords)))

50


#### Removing Stopwords (Set Difference)

In [12]:
print(len(remove_stopwords(lex, stopwords)))

1529


## 2. Ngrams and Ngram Probabilities

[n-gram](https://en.wikipedia.org/wiki/N-gram) is a contiguous sequence of *n* items from a given sequence of text or speech. An n-gram model models sequences, notably natural languages, using the statistical properties of n-grams.

__Example__:

- character n-grams: cat
- word n-grams: the cat is fat


|                     | 1-gram  | 2-gram  | 3-gram  |
|---------------------|---------|---------|---------|
|                     | unigram | bigram  | trigram |
| *Markov Order*      | 0       | 1       | 2       |
| *Character N-grams* | `['c', 'a', 't']` | `['ca', 'at']` | `['cat']` |
| *Word N-grams*      | `['the', 'cat', 'is' , 'fat']` | `['the cat', 'cat is', ...]` | `['the cat is', ...]` |


### 2.1. Counting Ngrams

*Frequency List* of a corpus is essentially a unigram count. Ngram count only differs in a unit of counting -- sequence of $n$ of words. We can compute bigram count by taking sequences of 2 items, trigrams - 3, etc.

#### 2.1.1. Data Structures
For *Frequency List* we have used [`dict`](https://docs.python.org/2/library/stdtypes.html#dict) to store counts.
In ngram counting scenario we still can use dictionary, and use ngram string keys; however, there are better data structures for the task. The frequent data structures used to store ngram counts or probabilities are [hash table](https://en.wikipedia.org/wiki/Hash_table), [trie](https://en.wikipedia.org/wiki/Trie), or [finite state automaton](https://en.wikipedia.org/wiki/Deterministic_acyclic_finite_state_automaton). The pros and cons of each data structure are out of the scope of this lab; for the discussion on efficient data structures for language modeling please refer to [KenLM paper](https://kheafield.com/papers/avenue/kenlm.pdf).

#### 2.1.2. Implementing Trie
There are plenty of implementation of the Trie data structure (e.g. [`pygtrie`](https://github.com/google/pygtrie/) from Google). However, it is simple enough. 

For understanding purposes let's implement our own, such that:

- works on lists of words
- stores in a node word, count, and children (`Node` class)
- implements methods to:
    - add a sequence (updating counts)
    - get a node by a sequence (i.e. prefix)
    - set a node attribute given sequence, key and value
    - traverse a trie and get all sequences
    
It is convenient to introduce an `error` node to the Trie to easily handle sequences not in it.
Thus, we also add `self.error` with `count` set to $0$.

In [13]:
class Node(object):

    def __init__(self, word=None):
        self.word = word
        self.children = {}
        self.count = 0
        
    def __set__(self, instance, value):
        self.instance = value

    def __get__(self, instance, owner):
        return self.instance


class Trie(object):
    
    def __init__(self):
        self.root = Node('*')
        self.error = Node()
    
    # convenient to set and get values (e.g. model metadata)
    def __set__(self, instance, value):
        self.instance = value

    def __get__(self, instance, owner):
        return self.instance

    def add(self, sequence):
        node = self.root
        node.count += 1  # total count
        for word in sequence:
            node.children[word] = node.children.setdefault(word, Node(word))
            node = node.children[word]
            node.count += 1

    def get(self, sequence):
        node = self.root
        for word in sequence:
            if not node.children:
                return self.error
            elif not word in node.children:
                return self.error
            else:
                node = node.children.get(word)
        return node
    
    def traverse(self, node=None, sequence=[]):
        node = self.root if not node else node
        
        if not node.children:
            yield sequence

        for word, n in node.children.items():
            sequence.append(word)
            yield from self.traverse(n, sequence)
            sequence.pop()

##### Testing out the implementation...

In [14]:
counts = Trie()
counts.add(['a', 'b', 'c', 'd'])
counts.add(['a', 'e', 'c', 'f'])

counts.size = 4

tests = [['a'], ['a', 'b'], ['a', 'x']]

for seq in tests:
    print(counts.get(seq).count)

for seq in counts.traverse():
    print(seq)
    
print(counts.size)

2
1
0
['a', 'b', 'c', 'd']
['a', 'e', 'c', 'f']
4


#### Exercise

- define a function to extract ngrams (variable $n$) from a sequence (sentence) represented as a list
- define a function to compute ngram counts from a corpus (list-of-lists) and store as a Trie
- compute bigram counts for the training set of NL2SparQL4NLU
- report `5` most frequent bigrams comparing to the reference values below (you can use `nbest` on `node.children`)


 word 1 | word 2 | count 
:-------|:-------|-------:
show    | me     |   377
the     | movie  |   267
of      | the    |   186
me      | the    |   122
is      | the    |   120


### 2.2. Computing Ngram Probabilities

#### 2.2.1. Calculating Probability from Frequencies

Probabilities of ngrams can be computed *normalizing* frequency counts (*Maximum Likelihood Estimation*): dividing the frequency of an ngram sequence by the frequency of its prefix (*relative frequency*).

N-gram   | Equation                      
:--------|:------------------------------
Unigram  | $$p(w_i) = \frac{c(w_i)}{N}$$ 
Bigram   | $$p(w_i | w_{i-1}) = \frac{c(w_{i-1}, w_i)}{c(w_{i-1})}$$ 
Ngram    | $$p(w_i | w_{i-n+1}^{i-1}) = \frac{c(w_{i-n+1}^{i-1}, w_i)}{c(w_{i-n+1}^{i-1})}$$ 

where:
- $N$ is the total number of words in a corpus
- $c(x)$ is the count of occurences of $x$ in a corpus (x could be unigram, bigram, etc.)

##### Exercise

- define a function to compute ngram probabilities using ngram counts

- compute probabilities of ngrams ($n=2$) in the training set of NL2SparQL4NLU
- report probabilities of the following ngrams
    - $p(the | of)$
    - $p(the | is)$
    - $p(play | the)$
    - probabilities of all bigram where $w_1$ is "*italy*", i.e. $p(*|italy)$

#### 2.2.2. Underflow Problem

Probabilities are usually small ($<1$).
Multiplying many of those may cause *underflow* problem.

Use the sum of the probabilities' logs instead of product

| Properties     
|:---------------
| $$p(a) > p(b)$$
| $$log(p(a)) > log(p(b))$$
| $$log(a*b) = log(a) + log(b)$$
| $$p(a) * p(b) \rightarrow log(p(a)) + log(p(b))$$


##### Exercise
- update the function to compute probabilities to use log (use `math` library)
- define a function to convert log probabilities to probabilities (`exp()`)

### <span style="color: blue;">Solutions</span>

#### Function Definitions

In [15]:
def ngrams(sequence, n=2):
    """
    returns ngrams as a list-of-lists of sequence elements
    :param sequence: list of elements
    :param n: ngram size to extract
    :return: list of ngrams
    """
    return [sequence[i:i+n] for i in range(len(sequence) - n + 1)]

def ngramcount(corpus, n=2):
    """
    count ngrams in a corpus and stores as a Trie
    :param corpus: list-of-lists
    :param n: ngram size to count
    :param glue: symbol for ngram concatenation into string
    :return: dict of ngram frequencies
    """
    counts = Trie()
    counts.size = n  # meta-info: ngram-size
    for sent in corpus:
        for ngram in ngrams(sent, n=n):
            counts.add(ngram) 
    return counts

def ngramprobs(counts, logs=True):
    """
    compute ngram probabilities from frequency counts
    :param counts: counts trie
    :param logs: if to compute log-probabilities
    :return: trie augmented with probabilties
    """
    from math import log
    
    # update default values:
    counts.error.probability = 0.0  # set error node probability to 0
    counts.logs = logs  # meta-info: log probabilities are used
    
    for ngram in counts.traverse(): 
        n = counts.get(ngram)       # get ngram node
        p = counts.get(ngram[:-1])  # get parent node
        prob = n.count / p.count
        n.probability = log(prob) if logs else prob
    return counts

def logp2p(value):
    from math import exp
    return exp(value) if value else 0.0

#### Counting ngrams & Reporting most frequent

In [16]:
counts = ngramcount(corpus, n=2)

print(nbest({"+".join(ngram): counts.get(ngram).count for ngram in counts.traverse()}, n=5))

{'show+me': 377, 'the+movie': 267, 'of+the': 186, 'me+the': 122, 'is+the': 120}


#### Computing Probabilities & Querying Ngrams

In [17]:
raw_probs = ngramprobs(counts, logs=False)

print(raw_probs.get(['of', 'the']).probability)
print(raw_probs.get(['is', 'the']).probability)
print(raw_probs.get(['the', 'play']).probability)  # not in training data

for ngram in raw_probs.traverse(node=raw_probs.get(['italy']), sequence=['italy']):
    print(ngram, raw_probs.get(ngram).probability)
    
# testing logp2p
log_probs = ngramprobs(counts)

print(logp2p(log_probs.get(['of', 'the']).probability))
print(logp2p(log_probs.get(['is', 'the']).probability))
print(logp2p(log_probs.get(['the', 'play']).probability))  # not in training data

0.30642504118616143
0.36363636363636365
0.0
['italy', 'make'] 0.5
['italy', 'in'] 0.5
0.30642504118616143
0.36363636363636365
0.0


## 3. Using Ngram Language Models

A statistical [language model](https://en.wikipedia.org/wiki/Language_model) is a probability distribution over sequences of words. Given such a sequence, say of length $n$, it assigns a probability $p(w_{1},\ldots ,w_{n})$ ($p(w_{1}^{n})$, for compactness) to the whole sequence (using Chain Rule). Consequently, the unigram and bigram probabilities computed above constitute an ngram language model of our corpus.

It is more useful for Natural Language Processing to have a __probability__ of a sequence being legal, rather than a grammar's __boolean__ decision whether it is legal or not.

### 3.1. Computing Probability of a Sequence (Scoring)

The most common usage of a language model is to compute probability of a sequence.

#### 3.1.1. Probability of a Sequence: [Chain Rule](https://en.wikipedia.org/wiki/Chain_rule_(probability))

Probability of a sequence is computed as a product of conditional probabilities (chain rule). 

$$p(w_{1}^{n}) = p(w_1) p(w_2|w_1) p(w_3|w_1^2) ... p(w_n|w_{1}^{n-1}) = \prod_{i=1}^{n}{p(w_i|w_{1}^{i-1})}$$

The order of ngram makes a simplifying assumption that probability of a current word only depends on previous $N - 1$ elements. Thus, it truncates previous context (history) to length $N - 1$.

$$p(w_i|w_{1}^{i-1}) \approx p(w_i|w_{i-N+1}^{i-1})$$

Consequently we have:

N-gram   | Equation                   |
:--------|:---------------------------|
unigram  | $$p(w_i)$$                 |
bigram   | $$p(w_i|w_{i-1})$$         |
trigram  | $$p(w_i|w_{i-2},w_{i-1})$$ |

The probability of the whole sequence applying an ngram model becomes:

$$p(w_{1}^{n}) = \prod_{i=1}^{n}{p(w_i|w_{i-N+1}^{i-1})}$$

#### 3.1.2. Sentence beginning & end tags

Including sentence boundary markers leads to a better model. To do that we need to augment each sentence with a special symbols for beginning and end of sentence tags (`<s>` and `</s>`, respectively). The beginning of the sentence tag gives the bigram context of the first word; and encodes probability of a word to start a sentence. Adding the end of the sentence tag, on the other hand, makes the bigram model a true probability distribution (Jurafsky and Martin). "Without it, the sentence probabilities for all sentences of a given length would sum to one. This model would define an infinite set of probability distributions, with one distribution per sentence length."

For larger ngrams, we’ll need to assume extra context for the contexts to the left and right of the sentence boundaries. For example, to compute trigram probabilities at the very beginning of the sentence, we can use two pseudo-words for the first trigram (i.e. `['<s>', '<s>', w1]`). Alternatively, we can use [back-off](https://en.wikipedia.org/wiki/Katz%27s_back-off_model), and use the `['<s>', w1]` bigram probability. 

__Example__: "The cat is fat"

$$p(\langle s \rangle, the, cat, is, fat, \langle /s \rangle) =$$
$$= p(the | \langle s \rangle) * p(cat | the) * p(is | cat) * p(fat | is) * p(\langle /s \rangle | fat) = $$
$$= 0.25 * 0.10 * 0.20 * 0.05 * 0.15 = 0.0000375$$

##### Exercise
- define a function to add sentence beginning and end tags to a corpus as list-of-lists
- define a function to compute sentence probability given an ngram model
    - remember that for log we use sum for raw probabilities product
        - use `math.prod` for Python 3.8
        - use `numpy.prod` for Python < 3.8
- re-compute bigram probabilities for the training set of NL2SparQL4NLU
- compute probability of sentences: *star of twilight* and *star of thor*

### 3.2. Generating Sequences

Ngram Model can be used as an automaton to generate probable legal sequences using the algorithm below.

__Algorithm for Bigram LM__

- $w_{i-1} = \langle s \rangle$;
- *while* $w_i \neq \langle /s \rangle$

    - stochastically get new word w.r.t. $p(w_i|w_{i-1})$

#### Exercise
- define a function to generate random sentences (e.g. using `random.choice`)
- generate `5` different sentences & score them

### 3.3. Evaluation: [Perplexity](https://en.wikipedia.org/wiki/Perplexity)

- Measures how well model fits test data
- Probability of test data
- Weighted average branching factor in predicting the next word (lower is better).
- Computed as:

$$ PPL = \sqrt[N]{\frac{1}{p(w_1,w_2,...,w_N)}} = \sqrt[N]{\frac{1}{\prod_{i=1}^{N}p(w_i|w_{i-N+1})}}$$

Where $N$ is the number of words in test set.

### <span style="color: blue;">Solutions</span>

#### Defined Functions

In [18]:
def corpus_add_tags(corpus, bos='<s>', eos='</s>'):
    """
    add beginning-of-sentence (bos) and end-of-sentence (eos) tags
    :param corpus: corpus as list-of-lists
    :param bos: beginning-of-sentence tag
    :param eos: end-of-sentence tag
    """
    return [[bos] + sent + [eos] for sent in corpus]

def score(model, sentence):
    """
    score a sentence given ngram model
    :param model: trie ngram model
    :param sentence: sentence as a list of tokens
    :return: log probability
    """
    from numpy import prod
    probs = [model.get(ngram).probability for ngram in ngrams(sentence, model.size)]
    return sum(probs) if model.logs else prod(probs)
    
def generate(model, bos='<s>', eos='</s>'):
    """
    generate a random sequence from ngram model
    :param model: trie ngram model
    :param bos: beginning-of-sentence tag
    :param eos: end-of-sentence tag
    :return: sentence as list & log probability
    """
    import random
    word = bos
    sent = [bos]
    while word != eos:
        c_node = model.get(sent[-(model.size - 1):])
        word = random.choice(list(c_node.children.keys()))
        sent.append(word)
    return sent

#### Scoring Sentences

In [19]:
# tag corpus
tagged_corpus = corpus_add_tags(corpus)

# re-build ngram model
tagged_counts = ngramcount(tagged_corpus, n=2)
tagged_probs = ngramprobs(tagged_counts)

# sentence adding tags
sent1 = corpus_add_tags([['star', 'of', 'twilight']])[0]
sent2 = corpus_add_tags([['star', 'of', 'thor']])[0]

print(score(tagged_probs, sent1))
print(score(tagged_probs, sent2))

-15.643604864128623
-9.609769522510536


#### Generating Sentences

In [27]:
for i in range(5):
    s = generate(tagged_probs)
    print("({:5.2f}): {}".format(score(tagged_probs, s), " ".join(s)))

(-10.60): <s> gross overall </s>
(-8.11): <s> auqua fish </s>
(-16.13): <s> which company for pulp fiction </s>
(-10.95): <s> zombie apocolypse </s>
(-25.03): <s> italian movies he was amazing </s>


### 3.4. Handling Unseen Words

Out-Of-Vocabulary (OOV) word -- tokens in test data that are not contained in the lexicon (vocabulary).
Empirically each OOV word results in 1.5 - 2 extra errors (> 1 due to the loss of contextual information).

__*How to handle words (in test set) that were never seen in the training data?*__

Train a language model with specific token (e.g. `<unk>`) for unknown words!

__*How to estimate probabilities of unknown words and ngrams?*__

The *simplest* approach is to replace all the words that are not in vocabulary (lexicon) with the `<unk>` token and treat it as any other word. (For instance, applying frequency cut-off to the lexicon, will allow estimate these probabilities on the training set.)

#### Exercise
- define a function to replace OOV words in a corpus as list-of-list given a lexicon
- re-compute bigram probabilities for the training set of NL2SparQL4NLU (with `<unk>`)
- re-compute probability of sentences: *star of twilight* and *star of thor*
    - replace unknown words and add tags as well
- compare scores to the ones without unknown word handling

#### <span style="color: blue;">Solutions</span>

#### Defined Functions

In [21]:
def corpus_replace_oov(corpus, lexicon, unk='<unk>'):
    """
    replace all tokens that are not in lexicon with unk
    :param corpus: corpus as list-of-lists
    :param lexicon: lexicon as a list of tokens
    :return: processed corpus
    """
    return [[token if token in lexicon else unk for token in sent] for sent in corpus]

#### Replacing OOV & re-scoring

In [22]:
# replace unknown corpus
unk_lexicon = cutoff(compute_frequency_list(tagged_corpus), tf_min=2)
unk_corpus = corpus_replace_oov(tagged_corpus, unk_lexicon)

# re-build ngram model
unk_counts = ngramcount(unk_corpus, n=2)
unk_probs = ngramprobs(unk_counts)

# sentence adding tags
sent1 = corpus_replace_oov(corpus_add_tags([['star', 'of', 'twilight']]), unk_lexicon)[0]
sent2 = corpus_replace_oov(corpus_add_tags([['star', 'of', 'thor']]), unk_lexicon)[0]

print(score(unk_probs, sent1))
print(score(unk_probs, sent2))

-15.643604864128623
-13.731498255275248


### 3.6. Handling Data Sparseness

What do we do with words that are in our lexicon, but appear in a test set in an unseen context (i.e. no ngram)?

Similar to unseen words and unseen n-grams have $0$ probability; thus, whole sequence gets $0$ probability.
(The problem is somewhat avoided using log probabilities.)

Use smoothing:
- Add some probability to unseen events
- Remove some probability from seen events
- Joint probability distribution sums to 1!

##### Smoothing Methods

Available smoothing methods: ([tutorial](https://nlp.stanford.edu/~wcmac/papers/20050421-smoothing-tutorial.pdf))
- [Additive smoothing](https://en.wikipedia.org/wiki/Additive_smoothing) (__simplest__)
- Good-Turing estimate
- Jelinek-Mercer smoothing (interpolation)
- Katz smoothing (backoff)
- Witten-Bell smoothing
- Absolute discounting
- Kneser-Ney smoothing


#### 3.6.1. Add-One Smoothing
Kind of Additive Smoothing.

__Bigrams__

$V$ -- vocabulary size

$$p(w_i | w_{i-1}) = \frac{c(w_{i-1},w_i)+1}{c(w_{i-1})+V}$$

__N-grams__

$V$ -- total number of possible $(N-1)$-grams

$$p(w_i | w^{i-1}_{i-N+1}) = \frac{c(w^{i-1}_{i-N+1},w_i)+1}{c(w^{i-1}_{i-N+1})+V}$$

Typically, we assume $V = \{w : c(w) > 0\} \cup \{<unk>\}$


#### Exercise
- update the ngram probability calculation function to apply add one smoothing
- re-compute bigram probabilities for the training set of NL2SparQL4NLU with smoothing
- compare scores to the ones without smoothing

#### <span style="color: blue;">Solutions</span>

#### Defined Functions

In [28]:
def compute_ngram_vocabulary_size(counts, n=1):
    return len(set(["+".join(ngram[:n]) for ngram in counts.traverse()]))

def ngramprobs(counts, logs=True, smoothing=False):
    """
    compute ngram probabilities from frequency counts
    :param counts: counts trie
    :param logs: if to compute log-probabilities
    :param smoothing: if to use add 1 smoothing
    :return: trie augmented with probabilties
    """
    from math import log
    
    # set meta-information
    counts.logs = logs  # meta-info: log probabilities are used
    counts.smoothing = smoothing # meta-info: smoothing true|false
    
    # add 1 smoothing
    v = compute_ngram_vocabulary_size(counts, n = counts.size - 1) if smoothing else 0
    a = 1 if smoothing else 0
    
    # update error probability:
    counts.error.probability = log(a / v) if (smoothing and logs) else 0.0
    
    for ngram in counts.traverse(): 
        n = counts.get(ngram)       # get ngram node
        p = counts.get(ngram[:-1])  # get parent node
        prob = (n.count + a) / (p.count + v)
        n.probability = log(prob) if logs else prob
    return counts

In [29]:
# re-build ngram model
sm_probs = ngramprobs(unk_counts, smoothing=True)

# sentence adding tags
sent1 = corpus_replace_oov(corpus_add_tags([['star', 'of', 'twilight']]), unk_lexicon)[0]
sent2 = corpus_replace_oov(corpus_add_tags([['star', 'of', 'thor']]), unk_lexicon)[0]

print(score(unk_probs, sent1))
print(score(unk_probs, sent2))

-23.038581629648416
-17.936125948586973


### 3.7. Putting it all together (Exercise)

Train an Ngram Language Model (compute ngram probabilities) such that:

- case insensitive (by default)
- 2-gram
- log probabilities
- considers sentence boundaries (beginning and end of sentence tags)
- considers unknown words
- Add-One Smoothing

Compute probabilties of utterances in `NL2SparQL4NLU/dataset/NL2SparQL4NLU.test.utterances.txt`

#### <span style="color: blue;">Solution</span>

In [25]:
trn='NL2SparQL4NLU/dataset/NL2SparQL4NLU.train.utterances.txt'
tst='NL2SparQL4NLU/dataset/NL2SparQL4NLU.test.utterances.txt'

trn_data_raw = read_corpus(trn)
trn_data_tag = corpus_add_tags(trn_data_raw)

# make lexicon
lex = cutoff(compute_frequency_list(tagged_corpus), tf_min=2)

trn_data_unk = corpus_replace_oov(trn_data_tag, lex)

# re-build ngram model
counts = ngramcount(trn_data_unk, n=2)
probs = ngramprobs(counts, logs=True, smoothing=True)

# prepare test set
tst_data_raw = read_corpus(tst)
tst_data_tag = corpus_add_tags(tst_data_raw)
tst_data_unk = corpus_replace_oov(tst_data_tag, lex)

for sent in tst_data_unk:
    print("({:5.2f}): {}".format(score(probs, sent), " ".join(sent)))

(-17.94): <s> star of <unk> </s>
(-27.43): <s> who is in the movie the campaign </s>
(-34.34): <s> list the cast of the movie the campaign </s>
(-21.80): <s> who was in twilight </s>
(-16.45): <s> who is in <unk> </s>
(-17.64): <s> actor from lost </s>
(-15.91): <s> who played in the movie rocky </s>
(-32.10): <s> who played in the movie captain america </s>
(-29.33): <s> cast and crew for in july </s>
(-27.05): <s> who is in movie in july </s>
(-39.19): <s> who 's in star wars episode four </s>
(-24.22): <s> who was in apollo thirteen </s>
(-22.52): <s> who was apollo thirteen 's cast </s>
(-48.00): <s> search for information about the cast and crew of <unk> thirteen </s>
(-62.58): <s> i would like to know more about the cast and crew of the movie apollo thirteen </s>
(-22.86): <s> who starred in the avengers </s>
(-21.89): <s> cast for finding nemo </s>
(-18.44): <s> actor from <unk> </s>
(-21.89): <s> list actors in the movie <unk> <unk> </s>
(-14.24): <s> dirty dancing actors </s>
